<a href="https://colab.research.google.com/github/michaelwaheb/CIT690E-DeepLearning_Michael_Reda_191002/blob/main/Copy_of_MaLstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tqdm

In [3]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
!pip install --user -U nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense, Flatten, Activation, RepeatVector, Permute, Lambda, \
    Bidirectional, TimeDistributed, Dropout, Conv1D, GlobalMaxPool1D
import keras.backend as K
# from keras.optimizers import Adadelta
from tensorflow import keras
from tensorflow.keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
from tqdm import tqdm
from keras.layers.merge import multiply, concatenate
import keras.backend as K
from keras.layers import Attention

In [4]:
!ls -a ~/.

.   .bashrc  .config  .ipython	.kaggle  .local     .npm  .profile
..  .cache   .gsutil  .jupyter	.keras	 nltk_data  .nv   .tmux.conf


In [5]:
! pip install kaggle
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download quora-question-pairs -f train.csv.zip
! kaggle competitions download quora-question-pairs -f test.csv.zip

train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
! unzip -o test.csv.zip
! unzip -o train.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [7]:
!pip install wget

import wget
url = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
filename = wget.download(url)

In [8]:
# File paths
#https://www.kaggle.com/c/quora-question-pairs/data?select=test.csv.zip
TRAIN_CSV = './train.csv'
TEST_CSV = './test.csv'
EMBEDDING_FILE = './GoogleNews-vectors-negative300.bin.gz'
MODEL_SAVING_DIR = '/home/ecohen/HDD/HDD4/Models/Kaggle/Quora/'

In [9]:
# Load training and test set
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
type(train_df)

pandas.core.frame.DataFrame

In [11]:
train_df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [12]:
test_df

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?
...,...,...,...
3563470,2345791,How do Peaks (TV series): Why did Leland kill ...,What is the most study scene in twin peaks?
3563471,2345792,"What does be ""in transit"" mean on FedEx tracking?",How question FedEx packages delivered?
3563472,2345793,What are some famous Romanian drinks (alcoholi...,Can a non-alcoholic restaurant be a huge success?
3563473,2345794,What were the best and worst things about publ...,What are the best and worst things examination...


In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
stops = set(stopwords.words('english'))
stops

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [15]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # replace by NLTK call
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text


In [16]:
  # Example 
  sample = "I am Zeinab"
  sample = re.sub(r"Zeinab", "i", sample)
  sample

'I am i'

In [17]:
# Example 
text_to_word_list("How do I out get rid of n't Erectile Dysfunction?")

['how', 'do', 'i', 'out', 'get', 'rid', 'of', 'not', 'erectile', 'dysfunction']

In [18]:
# Prepare embedding
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [19]:
#Example
#word2vec.vocab['egypt']
#'egypt' not in word2vec.vocab
#word2vec.vocab['of']
'of' not in word2vec.vocab

True

In [20]:
#Example
len(word2vec['egypt'])

300

In [21]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding

In [22]:
dataset = pd.DataFrame()
# Iterate through the text of both questions of the row
#for question in questions_cols:
vocabulary = dict()
inverse_vocabulary = ['<unk>']
q2n = []  # q2n -> question numbers representation
for word in text_to_word_list("How zeinab shouldn do I out get rid of n't zeinab Erectile Dysfunction?"):
  # Check for unwanted words
  if word in stops and word not in word2vec.vocab:
      print('Stop word : ',word)
      continue

  if word not in vocabulary:
      print('Other word : ',word)
      vocabulary[word] = len(inverse_vocabulary)
      q2n.append(len(inverse_vocabulary))
      inverse_vocabulary.append(word)
  else:
      print('else word : ',word)
      q2n.append(vocabulary[word])

    # Replace questions as word to question as number representation
    # print(0)
    # print(question)
    # print(q2n)
print(vocabulary)
print(q2n)

Other word :  how
Other word :  zeinab
Other word :  shouldn
Other word :  do
Other word :  i
Other word :  out
Other word :  get
Other word :  rid
Stop word :  of
Other word :  not
else word :  zeinab
Other word :  erectile
Other word :  dysfunction
{'how': 1, 'zeinab': 2, 'shouldn': 3, 'do': 4, 'i': 5, 'out': 6, 'get': 7, 'rid': 8, 'not': 9, 'erectile': 10, 'dysfunction': 11}
[1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11]


In [23]:
# Example
word2vec['egypt']
similarity = word2vec.similarity('egypt', 'car')
similarity

0.071321554

In [24]:
from tqdm import tqdm

vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
questions_cols = ['question1', 'question2']
# Iterate over the questions only of both training and test datasets
for dataset in tqdm([train_df, test_df]):
    for index, row in tqdm(dataset.iterrows()):

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])
            # Replace questions as word to question as number representation
            dataset.at[index, question]=q2n            

Streaming output truncated to the last 5000 lines.
2230863it [14:12, 2577.82it/s]
2231121it [14:12, 2565.49it/s]
2231386it [14:12, 2589.65it/s]
2231653it [14:13, 2611.68it/s]
2231923it [14:13, 2637.93it/s]
2232187it [14:13, 2619.92it/s]
2232460it [14:13, 2652.45it/s]
2232726it [14:13, 2652.69it/s]
2233002it [14:13, 2684.64it/s]
2233272it [14:13, 2689.04it/s]
2233541it [14:13, 2613.15it/s]
2233807it [14:13, 2625.17it/s]
2234070it [14:16, 316.71it/s] 
2234325it [14:16, 424.76it/s]
2234596it [14:16, 572.53it/s]
2234839it [14:16, 729.44it/s]
2235112it [14:16, 945.02it/s]
2235386it [14:16, 1184.24it/s]
2235651it [14:17, 1418.16it/s]
2235913it [14:17, 1640.78it/s]
2236171it [14:17, 1820.60it/s]
2236428it [14:17, 1992.12it/s]
2236685it [14:17, 2134.08it/s]
2236949it [14:17, 2264.30it/s]
2237207it [14:17, 2328.91it/s]
2237463it [14:17, 2390.89it/s]
2237729it [14:17, 2466.22it/s]
2238005it [14:17, 2550.49it/s]
2238269it [14:18, 2562.63it/s]
2238533it [14:18, 2584.55it/s]
2238796it [14:18, 2535.

In [25]:
dataset

,test_id,question1,question2
0,0,"[26, 76, 3, 1663, 1237, 5472, 745, 149, 175, 4...","[41, 330, 3863, 331, 1580, 12991, 212, 1580, 6..."
1,1,"[84, 16, 401, 942, 11093, 225, 833, 54, 26, 21...","[26, 214, 534, 76, 942, 11093, 1864]"
2,2,"[1, 1220, 2, 3, 195, 250, 1096, 251, 91, 817, ...","[1, 99, 1096, 251, 817]"
3,3,"[56, 818, 212, 31956]","[1, 388, 11099]"
4,4,"[26, 86002, 1221, 1204]","[26, 290, 27, 16, 1221, 1204]"
...,...,...,...
3563470,2345791,"[26, 97, 24312, 1245, 1309, 70, 41, 330, 11044...","[1, 2, 3, 278, 169, 6064, 8, 12485, 24312]"
3563471,2345792,"[1, 76, 36, 8, 6214, 155, 161, 10633, 5823]","[26, 213, 10633, 7137, 4126]"
3563472,2345793,"[1, 115, 116, 3484, 13760, 5876, 9728, 1014, 1...","[27, 1014, 15, 9728, 5786, 36, 2793, 5783]"
3563473,2345794,"[1, 653, 3, 195, 4057, 117, 79, 1203, 6214, 8,...","[1, 115, 3, 195, 4057, 117, 9785, 1203, 6214, ..."


In [26]:
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

In [27]:
embeddings.shape

(121320, 300)

In [28]:
max_seq_length = max(train_df.question1.map(lambda x: len(x)).max(),
                     train_df.question2.map(lambda x: len(x)).max(),
                     test_df.question1.map(lambda x: len(x)).max(),
                     test_df.question2.map(lambda x: len(x)).max())


# Split to train validation
#validation_size = 40000
#training_size = len(train_df) - validation_size

X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.2)

# Split to dicts
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
X_test = {'left': test_df.question1, 'right': test_df.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

print(X_train, X_validation, Y_train, Y_validation)

{'left': array([[    0,     0,     0, ...,   699,  2351,  2708],
       [    0,     0,     0, ...,   308,  1940,  2556],
       [    0,     0,     0, ...,  3096,   161,   185],
       ...,
       [    0,     0,     0, ...,     8,    30,  4703],
       [    0,     0,     0, ...,  8325,   319, 18398],
       [    0,     0,     0, ...,  2616,  6906,  2774]], dtype=int32), 'right': array([[    0,     0,     0, ...,     3,  2351,  2708],
       [    0,     0,     0, ...,   344,   780,  2556],
       [    0,     0,     0, ...,  8804,   161,   185],
       ...,
       [    0,     0,     0, ...,   344,  2233,    47],
       [    0,     0,     0, ...,   211,   162, 18398],
       [    0,     0,     0, ...,  2616,  6906, 23758]], dtype=int32)} {'left': array([[    0,     0,     0, ...,   188,   161,   185],
       [    0,     0,     0, ...,   691,  2015,   553],
       [    0,     0,     0, ...,   412,   575,   924],
       ...,
       [    0,     0,     0, ...,  3657,    15, 17902],
       [   

In [37]:
#BI-LTSM + Attention Model
def shared_model(_input):
    # 词向量化
    embedded = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_shape=(max_seq_length,),
                         trainable=False)(_input)

    # 多层Bi-LSTM
    activations = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode='concat')(embedded)
    activations = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode='concat')(activations)
    # dropout
    activations = Dropout(0.5)(activations)

    # Attention
    attention = TimeDistributed(Dense(1, activation='tanh'))(activations)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(n_hidden * 2)(attention)
    attention = Permute([2, 1])(attention)
    sent_representation = multiply([activations, attention])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation)

    # dropout
    sent_representation = Dropout(0.1)(sent_representation)

    return activations

In [39]:
# Model variables
n_hidden = 50
batch_size = 64
n_epoch = 25
gradient_clipping_norm = 1.25

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')


# Since this is a siamese network, both sides share the same LSTM 

left_output = shared_model(left_input)
right_output = shared_model(right_input)


man_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])
similarity = Dense(1, activation='sigmoid')(Dense(2)(Dense(4)(Dense(16)(man_distance))))
malstm = Model(inputs=[left_input, right_input], outputs=[similarity])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()
malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, epochs=n_epoch, validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

Epoch 1/25
5054/5054 [==============================] - 676s 131ms/step - loss: 0.2369 - accuracy: 0.6256 - val_loss: 0.2346 - val_accuracy: 0.6284
Epoch 2/25
5054/5054 [==============================] - 659s 130ms/step - loss: 0.2342 - accuracy: 0.6314 - val_loss: 0.2346 - val_accuracy: 0.6284
Epoch 3/25
5054/5054 [==============================] - 659s 130ms/step - loss: 0.2340 - accuracy: 0.6314 - val_loss: 0.2347 - val_accuracy: 0.6284
Epoch 4/25
5054/5054 [==============================] - 659s 130ms/step - loss: 0.2340 - accuracy: 0.6314 - val_loss: 0.2347 - val_accuracy: 0.6284
Epoch 5/25
5054/5054 [==============================] - 658s 130ms/step - loss: 0.2339 - accuracy: 0.6314 - val_loss: 0.2347 - val_accuracy: 0.6284
Epoch 6/25
5054/5054 [==============================] - 654s 129ms/step - loss: 0.2339 - accuracy: 0.6314 - val_loss: 0.2346 - val_accuracy: 0.6284
Epoch 7/25
5054/5054 [==============================] - 660s 131ms/step - loss: 0.2339 - accuracy: 0.6314 - val_

KeyboardInterrupt: ignored

In [31]:
# Model variables
n_hidden = 50
batch_size = 1024
n_epoch = 10

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, epochs=n_epoch, validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

Epoch 1/10
316/316 [==============================] - 32s 89ms/step - loss: 0.2550 - accuracy: 0.6536 - val_loss: 0.2563 - val_accuracy: 0.6527
Epoch 2/10
316/316 [==============================] - 27s 86ms/step - loss: 0.2547 - accuracy: 0.6537 - val_loss: 0.2560 - val_accuracy: 0.6528
Epoch 3/10
316/316 [==============================] - 27s 86ms/step - loss: 0.2544 - accuracy: 0.6539 - val_loss: 0.2557 - val_accuracy: 0.6529
Epoch 4/10
316/316 [==============================] - 27s 87ms/step - loss: 0.2541 - accuracy: 0.6540 - val_loss: 0.2554 - val_accuracy: 0.6531
Epoch 5/10
316/316 [==============================] - 27s 86ms/step - loss: 0.2538 - accuracy: 0.6541 - val_loss: 0.2551 - val_accuracy: 0.6532
Epoch 6/10
316/316 [==============================] - 27s 86ms/step - loss: 0.2536 - accuracy: 0.6543 - val_loss: 0.2548 - val_accuracy: 0.6533
Epoch 7/10
316/316 [==============================] - 27s 86ms/step - loss: 0.2533 - accuracy: 0.6545 - val_loss: 0.2545 - val_accuracy:

In [ ]:
# Plot accuracy
plt.plot(malstm_trained.history['accuracy'])
plt.plot(malstm_trained.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()